# 🇺🇬 A Machine Learning Approach to Accurate Import Valuation in Uganda

This notebook aims to build predictive models for import valuation using Uganda's customs data (2020–2024). Inaccurate manual methods contribute to revenue leakage, and we aim to use ML to bridge that gap. This aligns with the goals of URA’s Vision 2040 and digital transformation.

**Objectives**:
- Predict CIF values accurately using supervised ML models
- Compare ML vs traditional valuation methods
- Provide visual insights for operational integration

---


In [4]:
#importing the required libraries
# === Essential Libraries ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
import warnings
import shap
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", lambda x: "%.2f" % x)


LOADING THE DATA

In [5]:
# Load training and testing datasets
df_train = pd.read_csv("Uganda_imports_train.csv")
df_test = pd.read_csv("Uganda_imports_test.csv")

In [7]:
# Overview
print(df_train.shape)
print(df_test.shape)

(70734, 26)
(70734, 25)


In [9]:
df_train.head(5)

,HS_Code,Item_Description,Country_of_Origin,Port_of_Shipment,Quantity,Quantity_Unit,Net_Mass_kg,Gross_Mass_kg,FOB_Value_USD,Freight_USD,...,Mode_of_Transport,Year,Month,Invoice_Amount,Valuation_Method,Value_per_kg,Value_per_unit,FOB_per_kg,Freight_per_kg,Insurance_per_kg
0,30049099,Generic pharmaceutical products,China,Port Bell,482.42,kg,2220.29,2403.43,2352.84,220.04,...,Water,2021,11,9671924.57,Deductive Value Method (DVM),4356.15,20048.76,1.06,0.10,0.02
1,30049099,Generic pharmaceutical products,China,Entebbe Airport,131.97,liters,348.67,377.42,2084.10,169.47,...,Air,2022,11,8412978.38,Computed Value Method (CVM),24128.77,63749.17,5.98,0.49,0.05
2,15079090,Vegetable fats and oils,Germany,Entebbe Airport,113.44,pairs,449.93,487.04,2759.84,151.30,...,Air,2022,3,10672562.76,Transaction Value of Similar Goods (TVSG),23720.50,94081.12,6.13,0.34,0.12
3,10063010,Milled rice,India,Busia,230.52,units,808.09,874.73,2917.65,214.86,...,Land,2023,4,11692581.49,Computed Value Method (CVM),14469.41,50722.63,3.61,0.27,0.05
4,84089010,Industrial machinery parts,Saudi Arabia,Entebbe Airport,341.70,boxes,896.63,970.58,6971.39,366.85,...,Air,2021,6,26519078.57,Computed Value Method (CVM),29576.39,77609.24,7.78,0.41,0.15


In [10]:
df_test.head(5)

,HS_Code,Item_Description,Country_of_Origin,Port_of_Shipment,Quantity,Quantity_Unit,Net_Mass_kg,Gross_Mass_kg,FOB_Value_USD,Freight_USD,...,Mode_of_Transport,Year,Month,Invoice_Amount,Valuation_Method,Value_per_kg,Value_per_unit,FOB_per_kg,Freight_per_kg,Insurance_per_kg
0,30049099,Generic pharmaceutical products,China,Port Bell,482.42,kg,2220.29,2403.43,2352.84,220.04,...,Water,2021,11,9671924.57,Deductive Value Method (DVM),4356.15,20048.76,1.06,0.10,0.02
1,30049099,Generic pharmaceutical products,China,Entebbe Airport,131.97,liters,348.67,377.42,2084.10,169.47,...,Air,2022,11,8412978.38,Computed Value Method (CVM),24128.77,63749.17,5.98,0.49,0.05
2,15079090,Vegetable fats and oils,Germany,Entebbe Airport,113.44,pairs,449.93,487.04,2759.84,151.30,...,Air,2022,3,10672562.76,Transaction Value of Similar Goods (TVSG),23720.50,94081.12,6.13,0.34,0.12
3,10063010,Milled rice,India,Busia,230.52,units,808.09,874.73,2917.65,214.86,...,Land,2023,4,11692581.49,Computed Value Method (CVM),14469.41,50722.63,3.61,0.27,0.05
4,84089010,Industrial machinery parts,Saudi Arabia,Entebbe Airport,341.70,boxes,896.63,970.58,6971.39,366.85,...,Air,2021,6,26519078.57,Computed Value Method (CVM),29576.39,77609.24,7.78,0.41,0.15


In [11]:
df_train.columns

Index(['HS_Code', 'Item_Description', 'Country_of_Origin', 'Port_of_Shipment',
       'Quantity', 'Quantity_Unit', 'Net_Mass_kg', 'Gross_Mass_kg',
       'FOB_Value_USD', 'Freight_USD', 'Insurance_USD', 'CIF_Value_USD',
       'CIF_Value_UGX', 'Unit_Price_UGX', 'Tax_Rate', 'Currency_Code',
       'Mode_of_Transport', 'Year', 'Month', 'Invoice_Amount',
       'Valuation_Method', 'Value_per_kg', 'Value_per_unit', 'FOB_per_kg',
       'Freight_per_kg', 'Insurance_per_kg'],
      dtype='object')

In [12]:
df_test.columns

Index(['HS_Code', 'Item_Description', 'Country_of_Origin', 'Port_of_Shipment',
       'Quantity', 'Quantity_Unit', 'Net_Mass_kg', 'Gross_Mass_kg',
       'FOB_Value_USD', 'Freight_USD', 'Insurance_USD', 'CIF_Value_USD',
       'CIF_Value_UGX', 'Tax_Rate', 'Currency_Code', 'Mode_of_Transport',
       'Year', 'Month', 'Invoice_Amount', 'Valuation_Method', 'Value_per_kg',
       'Value_per_unit', 'FOB_per_kg', 'Freight_per_kg', 'Insurance_per_kg'],
      dtype='object')

### OBSERVATION
Although the train and test datasets are structurally consistent—with 24 features—there is a key omission in the test data: the target variable Unit_Price_UGX. This is an intentional and standard practice in supervised machine learning, ensuring that the model is evaluated on truly unseen data. The consistent feature set between both datasets supports model generalization, while the exclusion of the target from the test set prevents data leakage. However, it is critical to ensure that all preprocessing steps (e.g., encoding, scaling, and imputation) applied during training are identically replicated on the test data to maintain prediction accuracy and integrity.

In [13]:
# proceeding to work more with the training dataset
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70734 entries, 0 to 70733
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   HS_Code            70734 non-null  int64  
 1   Item_Description   70734 non-null  object 
 2   Country_of_Origin  70734 non-null  object 
 3   Port_of_Shipment   70734 non-null  object 
 4   Quantity           70734 non-null  float64
 5   Quantity_Unit      70734 non-null  object 
 6   Net_Mass_kg        70734 non-null  float64
 7   Gross_Mass_kg      70734 non-null  float64
 8   FOB_Value_USD      70734 non-null  float64
 9   Freight_USD        70734 non-null  float64
 10  Insurance_USD      70734 non-null  float64
 11  CIF_Value_USD      70734 non-null  float64
 12  CIF_Value_UGX      70734 non-null  float64
 13  Unit_Price_UGX     70734 non-null  float64
 14  Tax_Rate           70734 non-null  float64
 15  Currency_Code      70734 non-null  object 
 16  Mode_of_Transport  707

Train Dataset Overview

The training dataset comprises 70,734 fully populated records across 26 columns, exhibiting no missing values—an ideal condition for robust modeling. The feature set includes a balanced mix of numerical (19 columns: 16 floats, 3 integers) and categorical (7 object-type) variables, which supports diverse predictive modeling techniques. The presence of rich transactional data—such as FOB_Value_USD, CIF_Value_UGX, and Invoice_Amount—alongside contextual descriptors like Country_of_Origin, Port_of_Shipment, and Valuation_Method, provides a strong foundation for capturing both quantitative and qualitative influences on the target variable Unit_Price_UGX. The absence of nulls reduces preprocessing overhead and points to a well-curated dataset, though proper encoding and scaling will still be required for the categorical and continuous variables, respectively, to ensure optimal model performance.

In [14]:
# checking for descriptive stats on the training dataset
df_train.describe()

,HS_Code,Quantity,Net_Mass_kg,Gross_Mass_kg,FOB_Value_USD,Freight_USD,Insurance_USD,CIF_Value_USD,CIF_Value_UGX,Unit_Price_UGX,Tax_Rate,Year,Month,Invoice_Amount,Value_per_kg,Value_per_unit,FOB_per_kg,Freight_per_kg,Insurance_per_kg
count,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00,70734.00
mean,41919932.31,316.51,725.91,785.78,4306.91,330.52,47.08,4685.70,17330989.04,151323.88,0.16,2022.00,6.53,17330989.04,104255.20,152333.30,25.92,1.98,0.29
std,29558572.64,306.91,746.19,807.74,1836.71,237.02,31.00,2006.42,7435593.62,163745.99,0.07,1.41,3.45,7435593.62,200483.72,162888.30,49.83,4.48,0.63
min,10063010.00,5.25,2.42,2.61,43.47,1.77,0.25,49.93,185312.63,62.20,0.05,2020.00,1.00,185312.63,248.31,692.50,0.06,0.00,0.00
25%,15079090.00,77.90,151.88,164.41,2942.24,138.34,22.06,3200.72,11828281.51,31865.05,0.10,2021.00,4.00,11828281.51,14080.66,32822.00,3.49,0.22,0.03
50%,30049099.00,183.18,391.19,423.45,4203.45,281.02,40.76,4566.11,16853980.07,86718.70,0.18,2022.00,7.00,16853980.07,40541.90,86718.70,10.08,0.63,0.09
75%,84089010.00,467.95,1099.81,1190.51,5548.97,474.60,65.84,6037.45,22333531.29,214177.60,0.20,2023.00,10.00,22333531.29,109842.08,214112.97,27.31,1.91,0.28
max,87032319.00,938.51,2220.29,2403.43,9679.16,1011.14,135.63,10532.16,38961476.57,729221.44,0.25,2024.00,12.00,38961476.57,3559753.60,729221.44,882.13,115.51,14.58


## Key Statistical Observations from Train Dataset
Data Distribution: Most numerical features show a right-skewed distribution, as evidenced by their means being higher than the medians (e.g., Unit_Price_UGX, Value_per_kg, FOB_per_kg). This suggests the presence of high-value outliers, which could influence model behavior and may require log transformation or robust scaling.

Target Variable (Unit_Price_UGX): The unit price in UGX ranges from 62.20 to 729,221.44, with a high standard deviation of 163,745.99, indicating substantial variance. Its median value (86,718.70) is significantly lower than the mean (151,323.88), confirming skewness.

Freight and Insurance Costs: Both Freight_USD and Insurance_USD exhibit wide variability, with maximum values exceeding 1,000 USD and 135 USD respectively. However, Freight_per_kg and Insurance_per_kg show that some shipments had minimal to no costs per kilogram, suggesting variable valuation or subsidization.

Mass and Quantity: The Gross_Mass_kg and Net_Mass_kg are highly correlated, with average values of 786 kg and 726 kg respectively. The mass-based price features (FOB_per_kg, Value_per_kg) are also spread out significantly, which may affect model sensitivity to weight-based features.

Time Span: The data spans from 2020 to 2024, which may allow for trend or seasonality analysis using the Year and Month columns. This temporal information can be used to detect policy changes, macroeconomic shifts, or demand cycles.

Taxes and Valuation: The Tax_Rate has a relatively small standard deviation and ranges between 5% and 25%, indicating tiered tax policies. This could be an influential categorical feature when paired with Valuation_Method.